In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Adapted from https://towardsdatascience.com/ensembling-convnets-using-keras-237d429157eb

In [ ]:
import tensorflow as tf
from keras.callbacks import History
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.datasets import cifar10
from keras.engine import training
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dropout, Activation, Average
from keras.losses import categorical_crossentropy
from keras.models import Model, Input
from keras.optimizers import Adam

import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization, Input, GlobalAveragePooling2D
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import InceptionResNetV2

import matplotlib.pyplot as plt


from keras.utils import to_categorical
from tensorflow.python.framework.ops import Tensor
from typing import Tuple, List
import glob
import numpy as np
import os
from keras.models import load_model
from keras.utils import to_categorical
from numpy import dstack

from sklearn.metrics import classification_report

import pandas as pd

In [ ]:
tf.random.set_seed(23)


IMG_SIZE_incres = 320
IMG_SIZE_effnet = 333
BATCH_SZ = 320

In [ ]:
effnet_path = "../input/effnetpp/eff.h5"
incRes_path = "../input/inceptionresnet/inceptionResNetv2_Sun_6pm.h5"

model_paths = [effnet_path, incRes_path]

list_of_models = [load_model(m_p) for m_p in model_paths]
effModel = list_of_models[0]
incResModel = list_of_models[1]

In [ ]:
train = pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")
train['label'] = train['label'].astype('string')
train.head()

diseases = pd.read_json("/kaggle/input/cassava-leaf-disease-classification/label_num_to_disease_map.json", typ='series')
diseases

In [ ]:
datagen_incres = image.ImageDataGenerator(rotation_range=360,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                brightness_range=[0.2,1.5],
                                shear_range=25,
                                zoom_range=0.3,
                                channel_shift_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1/255,
                                validation_split=0.15)

val_datagen_incres = image.ImageDataGenerator(rescale=1/255,
                                       validation_split = 0.2)

train_generator_incres = datagen_incres.flow_from_dataframe(
    dataframe=train,
    directory="/kaggle/input/cassava-leaf-disease-classification/train_images",
    x_col='image_id',
    y_col='label',
    target_size=(IMG_SIZE_incres, IMG_SIZE_incres),
    batch_size=32,
    subset='training',
    shuffle = True,
    class_mode='categorical'
)

val_generator_incres = val_datagen_incres.flow_from_dataframe(
    dataframe=train,
    directory="/kaggle/input/cassava-leaf-disease-classification/train_images",
    x_col='image_id',
    y_col='label',
    target_size=(IMG_SIZE_incres, IMG_SIZE_incres),
    batch_size=32,
    subset='validation',
    class_mode = 'categorical',
    shuffle = True
)

# Same as val_generator, except that shuffle = False
test_generator_incres = val_datagen_incres.flow_from_dataframe(
    dataframe=train,
    directory="/kaggle/input/cassava-leaf-disease-classification/train_images",
    x_col='image_id',
    y_col='label',
    target_size=(IMG_SIZE_incres, IMG_SIZE_incres),
    batch_size=BATCH_SZ,
    subset='validation',
    class_mode = 'categorical',
    shuffle = False
)

In [ ]:
datagen_effnet = image.ImageDataGenerator(rotation_range=360,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                brightness_range=[0.2,1.5],
                                shear_range=25,
                                zoom_range=0.3,
                                channel_shift_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1/255,
                                validation_split=0.15)

val_datagen_effnet = image.ImageDataGenerator(rescale=1/255,
                                       validation_split = 0.2)

train_generator_effnet = datagen_effnet.flow_from_dataframe(
    dataframe=train,
    directory="/kaggle/input/cassava-leaf-disease-classification/train_images",
    x_col='image_id',
    y_col='label',
    target_size=(IMG_SIZE_effnet, IMG_SIZE_effnet),
    batch_size=32,
    subset='training',
    shuffle = True,
    class_mode='categorical'
)

val_generator_effnet = val_datagen_effnet.flow_from_dataframe(
    dataframe=train,
    directory="/kaggle/input/cassava-leaf-disease-classification/train_images",
    x_col='image_id',
    y_col='label',
    target_size=(IMG_SIZE_effnet, IMG_SIZE_effnet),
    batch_size=32,
    subset='validation',
    class_mode = 'categorical',
    shuffle = True
)

# Same as val_generator, except that shuffle = False
test_generator_effnet = val_datagen_effnet.flow_from_dataframe(
    dataframe=train,
    directory="/kaggle/input/cassava-leaf-disease-classification/train_images",
    x_col='image_id',
    y_col='label',
    target_size=(IMG_SIZE_effnet, IMG_SIZE_effnet),
    batch_size=BATCH_SZ,
    subset='validation',
    class_mode = 'categorical',
    shuffle = False
)

Check that dataloaders working properly

In [ ]:
# test_imgs, labels = next(test_generator)
# print(test_imgs.shape)

# plt.figure(figsize=(20,10))
# for i in range(25):
#     plt.subplot(5,5,i+1)
#     plt.xticks([])
#     plt.yticks([])
#     plt.grid(False)
#     plt.imshow(test_imgs[i])
#     label1 = np.argmax(labels[i])
#     plt.xlabel(diseases.get(label1))
# plt.show()

testing with just one to make sure the averaging works


In [ ]:
#test_imgs_incres, labels_incres = next(test_generator_incres)
#test_imgs_effnet, labels_effnet = next(test_generator_effnet)

In [ ]:
#np.mean(labels_incres == labels_effnet)

In [ ]:
#test = iter(test_generator)

In [ ]:
#predictions = [incResModel.predict_on_batch(test_imgs_incres), effModel.predict_on_batch(test_imgs_effnet)]

In [ ]:
#avg_preds = np.average(predictions, axis=0)
#avg_preds

In [ ]:
#avg_preds.argmax(axis=1)

Now let's try scaling this up

In [ ]:
# test_labels = []
# test_prob_preds = []
# test_class_preds = []

# while True:
#     try:
#         imgs_incres, labels_incres = next(test_generator_incres)
#         imgs_effnet, labels_effnet = next(test_generator_effnet)
#     except StopIteration:
#         break   
#     else: # this is executed if the try clause does not raise an exception
#         test_labels.append(labels_incres)

#         predictions = [incResModel.predict_on_batch(imgs_incres), effModel.predict_on_batch(imgs_effnet)]
#         avg_preds = np.average(predictions, axis=0)
#         class_preds = avg_preds.argmax(axis=1)

#         test_prob_preds.append(avg_preds)
#         test_class_preds.append(class_preds)


In [ ]:
#test_acc = np.mean(test_class_preds == test_labels)
#test_acc

In [ ]:
import cv2

In [ ]:
sample_sub = pd.read_csv('/kaggle/input/cassava-leaf-disease-classification/sample_submission.csv')

def getIndividProbPred(image_str, IMG_SIZE, model):
    img = tf.keras.preprocessing.image.load_img('../input/cassava-leaf-disease-classification/test_images/' + image_str)
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.preprocessing.image.smart_resize(img, (IMG_SIZE, IMG_SIZE))
    img = tf.reshape(img, (-1, IMG_SIZE, IMG_SIZE, 3))
    prediction = model.predict(img/255)
    return prediction

def getIRpred(img_id): return getIndividProbPred(img_id, IMG_SIZE_incres, incResModel)
def getEffpred(img_id): return getIndividProbPred(img_id, IMG_SIZE_effnet, effModel)

def getEnsembPredForImg(img_id):
    predictions = [getIRpred(img_id), getEffpred(img_id)]
    avg_preds = np.average(predictions, axis=0)
    class_pred = avg_preds.argmax(axis=1)
    return class_pred.item(0)


In [ ]:
ss = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
preds = [getEnsembPredForImg(img_id) for img_id in ss.image_id]
preds

In [ ]:

submission = pd.DataFrame({'image_id': ss.image_id, 'label': preds})
submission

submission.to_csv('submission.csv', index = False)


In [ ]:
submission

In [ ]:
# labelNames = [cidx for cidx in train_generator.class_indices]

# predictions = []

# # loop over the models
# for model in list_of_models:
#      # use the current model to make predictions on the testing data,
#      # then store these predictions in the aggregate predictions list
#      predictions.append(model.predict(test_generator, batch_size=BATCH_SZ))
# # average the probabilities across all model predictions, then show
# # a classification report
# predictions = np.average(predictions, axis=0)

# print(classification_report(test_generator.classes,
#      predictions.argmax(axis=1), target_names=labelNames))

In [ ]:
#list_of_models[0].input_shape

In [ ]:

#labelNames = [cidx for cidx in train_generator.class_indices]
#print(classification_report(test_generator.classes,
#     predictions.argmax(axis=1), target_names=labelNames))

#print(classification_report(test_generator.classes, y_pred, target_names=labelNames))